In [385]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [386]:
df = pd.read_csv('car.csv')
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
df.columns = columns

In [387]:
df.head()


,buying,maint,doors,persons,lug_boot,safety,evaluation
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [388]:
df.shape

(1728, 7)

In [389]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   buying      1728 non-null   object
 1   maint       1728 non-null   object
 2   doors       1728 non-null   object
 3   persons     1728 non-null   object
 4   lug_boot    1728 non-null   object
 5   safety      1728 non-null   object
 6   evaluation  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [390]:
# df['Person_Capacity'] = LE.fit_transform(df['Person_Capacity'])
import warnings
warnings.filterwarnings("ignore")
mapping = {
    
    'm_buying_and_maint' : {
        # For 'buying' & 'maint' columns.
        'low': 1,
        'med': 2,
        'high': 3,
        'vhigh': 4,
    },
    
    'm_doors' : {
        # For 'doors' columns.
        '2': 1,
        '3': 2,
        '4': 3,
        '5more': 4,
    },
    
    'm_persons' : {
        # For 'persons' columns.
        '2': 1,
        '4': 2,
        'more': 3,
    },
    
    'm_lug_boot' : {
        # For 'lug_boot' column.
        'small': 1,
        'med': 2,
        'big': 3,
    },
    
    'm_safety' : {
        # For 'safety' column.
        'low': 1,
        'med': 2,
        'high': 3,
    },
    
    'm_evaluation' : {
        # For 'evaluation' column.
        'unacc': 1,
        'acc': 2,
        'good': 3,
        'vgood': 4
    }
}

df_encoded = df

df_encoded['buying'] = df['buying'].replace(mapping['m_buying_and_maint'])
df_encoded['maint'] = df['maint'].replace(mapping['m_buying_and_maint'])
df_encoded['doors'] = df['doors'].replace(mapping['m_doors'])
df_encoded['persons'] = df['persons'].replace(mapping['m_persons'])
df_encoded['lug_boot'] = df['lug_boot'].replace(mapping['m_lug_boot'])
df_encoded['safety'] = df['safety'].replace(mapping['m_safety'])
df_encoded['evaluation'] = df['evaluation'].replace(mapping['m_evaluation'])

df_encoded.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,4,4,1,1,1,1,1
1,4,4,1,1,1,2,1
2,4,4,1,1,1,3,1
3,4,4,1,1,2,1,1
4,4,4,1,1,2,2,1


In [391]:
df.dtypes

buying        int64
maint         int64
doors         int64
persons       int64
lug_boot      int64
safety        int64
evaluation    int64
dtype: object

In [392]:
df_encoded["evaluation"].value_counts()

evaluation
1    1210
2     384
3      69
4      65
Name: count, dtype: int64

In [393]:
# df = df[(df['evaluation'] == 1) | (df['evaluation'] == 2)]

In [394]:
A = df_encoded[df_encoded['evaluation'] == 4]
B = df_encoded[df_encoded['evaluation'] == 3]
C = df_encoded[df_encoded['evaluation'] == 2].head(67)
D = df_encoded[df_encoded['evaluation'] == 1].head(70)


In [395]:
df_encoded = pd.concat([A,B,C,D])
from sklearn.utils import shuffle

# Shuffle the DataFrame
df_encoded = df_encoded.sample(frac=1, random_state=42).reset_index(drop=True)

# `frac=1` ensures all rows are included in the shuffle.
# `reset_index(drop=True)` resets the index after shuffling.

df_encoded

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,1,3,2,3,3,3,4
1,1,1,2,2,1,3,3
2,2,1,3,3,1,3,3
3,1,1,3,3,3,2,3
4,4,1,4,2,1,3,2
...,...,...,...,...,...,...,...
266,2,1,3,3,2,3,4
267,4,1,3,2,2,3,2
268,2,1,2,2,2,3,3
269,1,2,4,2,2,2,3


In [396]:
X = df_encoded.iloc[:,:-1]
X

,buying,maint,doors,persons,lug_boot,safety
0,1,3,2,3,3,3
1,1,1,2,2,1,3
2,2,1,3,3,1,3
3,1,1,3,3,3,2
4,4,1,4,2,1,3
...,...,...,...,...,...,...
266,2,1,3,3,2,3
267,4,1,3,2,2,3
268,2,1,2,2,2,3
269,1,2,4,2,2,2


In [397]:
y = df_encoded.iloc[:,-1:]
y

,evaluation
0,4
1,3
2,3
3,3
4,2
...,...
266,4
267,2
268,3
269,3


In [398]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [399]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
RF = RandomForestClassifier()
from sklearn.model_selection import cross_val_score
from hyperopt import hp,tpe,fmin,STATUS_OK,Trials

In [400]:
RF = RandomForestClassifier(criterion = "gini", max_features =  "sqrt", n_estimators =   100)

In [401]:
RF.fit(X_train,y_train)


RandomForestClassifier()

In [402]:
y_hat = RF.predict(X_test)


In [403]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test,y_hat))


0.9818181818181818


In [415]:
n = 50
RF.predict([X_test.iloc[n]]), X_test.iloc[n].values, y_test.iloc[n]

(array([2], dtype=int64),
 array([4, 1, 3, 3, 3, 2], dtype=int64),
 evaluation    2
 Name: 15, dtype: int64)

In [405]:
RF.predict_proba([X_test.iloc[n]])

array([[1., 0., 0., 0.]])

In [406]:
print(classification_report(y_test,y_hat))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        14
           3       0.89      1.00      0.94         8
           4       1.00      0.94      0.97        17

    accuracy                           0.98        55
   macro avg       0.97      0.99      0.98        55
weighted avg       0.98      0.98      0.98        55



In [407]:
print(confusion_matrix(y_test,y_hat))


[[16  0  0  0]
 [ 0 14  0  0]
 [ 0  0  8  0]
 [ 0  0  1 16]]


In [408]:
RF

RandomForestClassifier()

In [409]:
import pickle
with open('RF.pkl', 'wb') as model_file:
    pickle.dump(RF, model_file)

In [410]:
RF.n_features_in_

6

In [411]:
import joblib
import onnx
import skl2onnx
from skl2onnx.convert import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load the .pkl model
model = joblib.load('RF.pkl')

# Convert the model to ONNX
onnx_model = convert_sklearn(model, initial_types=[('input', FloatTensorType([None, model.n_features_in_]))])

# Save the ONNX model
onnx.save_model(onnx_model, 'RF.onnx')


In [412]:
import onnx

# Load the ONNX model
model = onnx.load("RF.onnx")

# Print input names
print("Input names:")
for input in model.graph.input:
    print(input.name)

# Print output names
print("Output names:")
for output in model.graph.output:
    print(output.name)


Input names:
input
Output names:
output_label
output_probability


In [413]:
import onnx

# Load the ONNX model
model_path = "RF.onnx"  # Update with your actual model path
model = onnx.load(model_path)

# Get the model's input names
print("Input names:")
for input in model.graph.input:
    print(input.name)

# Get the model's output names
print("\nOutput names:")
for output in model.graph.output:
    print(output.name)


Input names:
input

Output names:
output_label
output_probability


In [416]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("RF.onnx")
inputs = {session.get_inputs()[0].name: np.array([[4,1 ,3 ,3, 3, 2]], dtype=np.float32)}
outputs = session.run(None, inputs)
print(outputs)


[array([2], dtype=int64), [{1: 0.0, 2: 0.9499999284744263, 3: 0.04999999701976776, 4: 0.0}]]


In [313]:
# scp -P 2222 -r /mnt/c/Users/A507658/Downloads/qwen2-docker/RF.onnx root@localhost:/root/

In [ ]:

# scp -P 2222 -r /mnt/c/Users/A507658/Downloads/qwen2-docker/classification_ann_model.onnx root@localhost:/root/

In [495]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import Counter

# Load the dataset
df = pd.read_csv('car.csv')
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'evaluation']
df.columns = columns

# Define the mappings for encoding categorical features
mapping = {
    'm_buying_and_maint': {
        'low': 1,
        'med': 2,
        'high': 3,
        'vhigh': 4,
    },
    'm_doors': {
        '2': 1,
        '3': 2,
        '4': 3,
        '5more': 4,
    },
    'm_persons': {
        '2': 1,
        '4': 2,
        'more': 3,
    },
    'm_lug_boot': {
        'small': 1,
        'med': 2,
        'big': 3,
    },
    'm_safety': {
        'low': 1,
        'med': 2,
        'high': 3,
    },
    'm_evaluation': {
        'unacc': 0,
        'acc': 1,
        'good': 2,
        'vgood': 3
    }
}

# Apply encoding
df_encoded = df.copy()
df_encoded['buying'] = df['buying'].replace(mapping['m_buying_and_maint'])
df_encoded['maint'] = df['maint'].replace(mapping['m_buying_and_maint'])
df_encoded['doors'] = df['doors'].replace(mapping['m_doors'])
df_encoded['persons'] = df['persons'].replace(mapping['m_persons'])
df_encoded['lug_boot'] = df['lug_boot'].replace(mapping['m_lug_boot'])
df_encoded['safety'] = df['safety'].replace(mapping['m_safety'])
df_encoded['evaluation'] = df['evaluation'].replace(mapping['m_evaluation'])

# Separate features and target
X = df_encoded.drop(['evaluation'], axis=1)
y = df_encoded['evaluation']

# Apply SMOTE to handle class imbalance
s = SMOTE(k_neighbors=2)
X_resampled, y_resampled = s.fit_resample(X, y)

# Print the class distribution after resampling
print(f'Total of labels: {len(y_resampled)}')
print(Counter(y_resampled))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# # Shift labels from [1, 2, 3, 4] to [0, 1, 2, 3]
# y_train = y_train - 1
# y_test = y_test - 1

# Define the ANN model for multi-class classification
def create_ann_model(input_dim, num_classes):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for multi-class classification
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Ensure y_train is one-hot encoded
num_classes = len(np.unique(y_train))
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)

# Create and compile the model
ann_model = create_ann_model(input_dim=X_train.shape[1], num_classes=num_classes)

# Train the model
ann_model.fit(X_train, y_train_one_hot, epochs=100, batch_size=32, verbose=1)

# Convert y_test to one-hot encoding for evaluation
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Predictions
predictions = np.argmax(ann_model.predict(X_test), axis=1)

# Evaluate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, predictions)
print("Confusion Matrix:\n", conf_matrix)

# Classification report
class_report = classification_report(y_test, predictions)
print("Classification Report:\n", class_report)


Total of labels: 4840
Counter({0: 1210, 1: 1210, 3: 1210, 2: 1210})
Epoch 1/100
121/121 [==============================] - 1s 1ms/step - loss: 0.8889 - accuracy: 0.6400
Epoch 2/100
121/121 [==============================] - 0s 987us/step - loss: 0.4550 - accuracy: 0.8675
Epoch 3/100
121/121 [==============================] - 0s 1ms/step - loss: 0.2566 - accuracy: 0.9354
Epoch 4/100
121/121 [==============================] - 0s 926us/step - loss: 0.1928 - accuracy: 0.9452
Epoch 5/100
121/121 [==============================] - 0s 1ms/step - loss: 0.1616 - accuracy: 0.9512
Epoch 6/100
121/121 [==============================] - 0s 1ms/step - loss: 0.1446 - accuracy: 0.9556
Epoch 7/100
121/121 [==============================] - 0s 1ms/step - loss: 0.1160 - accuracy: 0.9659
Epoch 8/100
121/121 [==============================] - 0s 1ms/step - loss: 0.1106 - accuracy: 0.9662
Epoch 9/100
121/121 [==============================] - 0s 930us/step - loss: 0.1031 - accuracy: 0.9675
Epoch 10/100
121/

In [497]:
import onnxmltools
import onnxruntime as ort

In [500]:
# Save the ANN model as ONNX
onnx_model = onnxmltools.convert_keras(ann_model)
onnxmltools.utils.save_model(onnx_model, "classification_ann_model.onnx")

# Load the ONNX model for inference using ONNX Runtime
ort_session = ort.InferenceSession("classification_ann_model.onnx")
# Example input for prediction
example_input1 = np.array([[ 4, 1, 3, 3, 3, 2]], dtype=np.float32)



# Prepare inputs for ONNX model inference
input_name = ort_session.get_inputs()[0].name
prediction1 = ort_session.run(None, {input_name: example_input1})[0]

# Print predictions from the ONNX model
print(f'Predicted MPG for input {example_input1}: {prediction1}')


Predicted MPG for input [[4. 1. 3. 3. 3. 2.]]: [[1.7787680e-06 9.9999821e-01 2.4764050e-17 1.0829036e-36]]


In [502]:
import numpy as np
import onnxruntime as ort

# Load the ONNX model for inference using ONNX Runtime
ort_session = ort.InferenceSession("classification_ann_model.onnx")

# Example input for prediction
example_input1 = np.array([[4, 1, 3, 3, 3, 2]], dtype=np.float32)

# Prepare inputs for ONNX model inference
input_name = ort_session.get_inputs()[0].name
prediction1 = ort_session.run(None, {input_name: example_input1})[0]

# Convert predictions to percentages
prediction_percentage = prediction1 * 100

# Print predictions as percentages for each class
for i, p in enumerate(prediction_percentage[0]):
    print(f"Class {i+1}: {p:.7f}%")


Class 1: 0.0001779%
Class 2: 99.9998245%
Class 3: 0.0000000%
Class 4: 0.0000000%


In [503]:
import onnx

# Load the ONNX model
model = onnx.load("classification_ann_model.onnx")

# Print input and output node names
print("Input names:")
for input in model.graph.input:
    print(input.name)

print("\nOutput names:")
for output in model.graph.output:
    print(output.name)

Input names:
dense_68_input

Output names:
dense_71


In [ ]:
aarch64-linux-gnu-g++ -o classification1 classification_ANN.cpp     -I/home/phani/onnxruntime-linux-aarch64-1.18.0/include     -L/home/phani/onnxruntime-linux-aarch64-1.18.0/lib     -lonnxruntime

In [ ]:
scp -P 2222 -r /mnt/c/Users/A507658/Downloads/qwen2-docker/classification_ann_model.onnx root@localhost:/root/